In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df=pd.read_csv('/kaggle/input/titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
#Data Preprocessing
def preprocess_data(df):
    #fill missing values
    df['Age'].fillna(df['Age'].mean(),inplace=True)  # replacing nan values with mean as it is a numerical column
    df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)  # As this is categorical column, replacing nan with mode(highest occuring value) and index [0] is used to consider the first occurance
    df['Fare'].fillna(df['Fare'].mean(),inplace=True)
    
    df=df.drop(['Name','Ticket','Cabin'],axis=1)
    
    # Encode categorical features
    le = LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])
    df['Embarked'] = le.fit_transform(df['Embarked'])
    #filling missing values should be done before encoding bcz if encoding is done first then nan is allocated with a number then nan can not be replaced
    
    return df

train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

# Split the data into features and labels
X = train_df.drop('Survived', axis=1).values
y = train_df['Survived'].values

In [6]:
# Preprocess the training data
#X_train, y_train = preprocess_data(train_data)
# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)


In [7]:
X_train_tensor.shape

torch.Size([712, 8])

In [8]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [9]:
# Define the Neural Network with Dropout
class TitanicMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(TitanicMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x
# Instantiate the model
input_size = X_train.shape[1] #Number of features
hidden_size = 64
output_size = 2  # Binary classification (Survived or not)
dropout_rate = 0.5  # Adjust as needed
model = TitanicMLP(input_size, hidden_size, output_size, dropout_rate)

In [10]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  #used for finding best weights

In [11]:
# Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # _, is used for storing temporary values
            total += labels.size(0)
            correct += (predicted == labels).sum().item()   
            val_loss += criterion(outputs, labels)
        avg_val_loss = val_loss / len(val_dataloader)
        accuracy = correct / total * 100.0

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

Epoch [1/15], Loss: 5.2253, Validation Loss: 3.7657, Accuracy: 56.98%
Epoch [2/15], Loss: 6.2039, Validation Loss: 2.8903, Accuracy: 58.66%
Epoch [3/15], Loss: 20.9218, Validation Loss: 1.8331, Accuracy: 62.01%
Epoch [4/15], Loss: 2.1482, Validation Loss: 1.9817, Accuracy: 61.45%
Epoch [5/15], Loss: 12.3598, Validation Loss: 1.6398, Accuracy: 63.13%
Epoch [6/15], Loss: 3.5160, Validation Loss: 1.2291, Accuracy: 63.69%
Epoch [7/15], Loss: 0.9414, Validation Loss: 0.6370, Accuracy: 72.07%
Epoch [8/15], Loss: 0.9232, Validation Loss: 0.9733, Accuracy: 63.69%
Epoch [9/15], Loss: 2.6269, Validation Loss: 0.8754, Accuracy: 63.69%
Epoch [10/15], Loss: 0.8624, Validation Loss: 0.5300, Accuracy: 71.51%
Epoch [11/15], Loss: 0.8705, Validation Loss: 0.6637, Accuracy: 64.25%
Epoch [12/15], Loss: 1.4908, Validation Loss: 0.7907, Accuracy: 62.01%
Epoch [13/15], Loss: 1.0350, Validation Loss: 0.5762, Accuracy: 74.30%
Epoch [14/15], Loss: 1.0337, Validation Loss: 0.6003, Accuracy: 73.18%
Epoch [15/15]

In [12]:
# Prediction on test set
X_test_tensor = torch.tensor(test_df.values, dtype=torch.float32)
test_dataset = TensorDataset(X_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
predictions = []
with torch.no_grad():
    for inputs in test_dataloader:
        outputs = model(inputs[0])
        _, predicted = torch.max(outputs, 1)  # _, is used for storing temporary values
        predictions.extend(predicted.numpy())  
    print("Predicted Labels:")
    print(predictions)

Predicted Labels:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [13]:
# Save predictions to a CSV file
submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
submission_df.to_csv('/kaggle/working/titanic_predictions.csv', index=False)